In [1]:
import ObservationObject

In [4]:
import astroquery


## Example Usage

This section focuses on all the possibilities that the Observation class introduces. First, let's create an Observation object for a specific content folder:

In [2]:
# Specifies the absolute path to our data directory
data_dir = "/net/vega/data/users/observatory/images/200417/STL-6303E/i/"

obs = Observation(data_dir)  # This line creates an instance of the Observation class

That's it! The Observation object performed some indexing under the hood at initialization, and is now ready for use. Let's take a look at how to use the Observation object for practical purposes and how it simplifies our lives:

In [3]:
# Let's see some properties of our data folder
number_files = len(obs.files)
print(f"Number of found .fit(s) files: {number_files}")

used_filters = obs.filters
print(f"Used filters: {used_filters}")

print("\n")

# Now, let's take a look at some light files and print a few header keywords
cool_files = obs.lightFiles
obs.print_content(files=cool_files[:8])

print("\n")

# Maybe we want to focus on some flat fields and their filter and binning only
cooler_files = obs.flatFiles
my_keywords = ["IMAGETYP", "FILTER", "XBINNING", "YBINNING"]
obs.print_content(files=cooler_files[:8], keywords=my_keywords)

print("\n")

# Perhaps we're really strict, and want to see only frames taken with an 1x1 binning in the V filter
my_conditions = {
    "XBINNING": 1,
    "YBINNING": 1,
    "FILTER": "V"
}
coolest_files = obs.get_files(condit=my_conditions)
obs.print_content(files=coolest_files, keywords=my_keywords)

Number of found .fit(s) files: 124
Used filters: ['B', 'V', 'I', 'R']


|                            Filename |      IMAGETYP |       EXPTIME |        FILTER |       IMGSIZE |
-------------------------------------------------------------------------------------------------------
|        200417_Li_.00000040.M_87.FIT |   Light Frame |         300.0 |             B |     3072x2048 |
|        200417_Li_.00000041.M_87.FIT |   Light Frame |         300.0 |             V |     3072x2048 |
|        200417_Li_.00000042.M_87.FIT |   Light Frame |         300.0 |             R |     3072x2048 |
|   ...7_Li_.00000043.Mosaic__A14.FIT |   Light Frame |         300.0 |             B |     3072x2048 |
|   ...7_Li_.00000044.Mosaic__A14.FIT |   Light Frame |         300.0 |             V |     3072x2048 |
|   ...7_Li_.00000045.Mosaic__A14.FIT |   Light Frame |         300.0 |             R |     3072x2048 |
|   ...17_Li_.00000046.Mosaic__A9.FIT |   Light Frame |         300.0 |             B |     3072

Now that we've had a taste of what our data is like, why not turn to frame correction immediately? There are multiple ways to get these frames:

In [4]:
# The most basic approach: ask the Observation instance to generate the master bias
masterBias = obs.masterBias
print(f"Master Bias:\n {masterBias} \n")

# Maybe we only have two competent dark frames, so let's create a master dark out of those two only
comp_darks = obs.darkFiles[3:5]
customMasterDark = obs.create_master_dark(darkFiles=comp_darks)
print(f"Master Dark:\n {customMasterDark} \n")

# Now it's time to create the master flat fields, but specifically for the B and I filters.
# Let's also pass our custom master dark here!
fltrs = ['B', 'I']
masterFlats = obs.create_master_flats(filterTypes=fltrs, masterDark=customMasterDark)

# masterFlats will be a 3D array, where flats are stacked on the z-axis. Let's loop over them:
for i in range(0, len(fltrs)):
    fltr = fltrs[i]
    masterFlat = masterFlats[:,:,i]
    print(f"Master Flat for {fltr}:\n {masterFlat} \n")

Master Bias:
 [[2038. 2040. 2040. ... 2014. 2012. 2010.]
 [2041. 2041. 2040. ... 2012. 2009. 2012.]
 [2043. 2042. 2040. ... 2011. 2009. 2012.]
 ...
 [2038. 2055. 2038. ... 2011. 2008. 2009.]
 [2038. 2058. 2037. ... 2006. 2010. 2009.]
 [2043. 2060. 2038. ... 2009. 2008. 2011.]] 

Master Dark:
 [[0.095      0.09666667 0.06       ... 0.09666667 0.095      0.06166667]
 [0.11166667 0.075      0.12166667 ... 0.06333333 0.12       0.04833333]
 [0.10833333 0.06333333 0.04666667 ... 0.07833333 0.10833333 0.09833333]
 ...
 [0.1        0.04666667 0.08166667 ... 0.075      0.06333333 0.06333333]
 [0.055      0.09333333 0.06166667 ... 0.09166667 0.08833333 0.08      ]
 [0.09166667 0.08833333 0.055      ... 0.065      0.10666667 0.05      ]] 

Master Flat for B:
 [[0.88114972 0.88112586 0.87574342 ... 0.86612744 0.85088874 0.82521146]
 [1.00056673 1.00574834 1.00612766 ... 0.97859822 0.99007892 0.96813668]
 [0.99207703 1.00659348 1.00937778 ... 0.98626465 0.97968388 0.96398956]
 ...
 [0.97873259 0.9

One of the great aspects of the Observation object is that it easily hooks into the Blaauw Archive:

In [5]:
from astroquery.utils.tap.core import Tap

from astropy.coordinates import SkyCoord
from astropy.table import Table
from astropy.time import Time

from matplotlib.pyplot import figure, show
import numpy as np

vo_url = "http://vo.astro.rug.nl/tap"
vo = Tap(url=vo_url, verbose=True)

# Get the JD version of Jan 1st of 2019
since_date = Time('2019-01-01T12:00:00.000').jd

# Prepare query (note, there is a 2000 row limit by default, use an async query above that)
query = """
SELECT TOP 50 filename, object, ra, dec, date_obs
FROM observations.raw
WHERE imagetyp = 'Light Frame' AND obs_jd >= {}
""".format(since_date)

job = vo.launch_job(query)
table = job.get_results()

filenames = [fn.decode() for fn in table['filename']]  # Note that we need to use .decode()

obs = Observation(filenames[0])  # Create the Observation object
obs.print_content()

Created TAP+ (v1.2.1) - Connection:
	Host: vo.astro.rug.nl
	Use HTTPS: False
	Port: 80
	SSL Port: 443
|                            Filename |      IMAGETYP |       EXPTIME |        FILTER |       IMGSIZE |
-------------------------------------------------------------------------------------------------------
|        190515_Li_00000001.BIAS.fits |    Bias Frame |           0.0 |             R |     3072x2048 |
|        190515_Li_00000002.BIAS.fits |    Bias Frame |           0.0 |             R |     3072x2048 |
|        190515_Li_00000003.BIAS.fits |    Bias Frame |           0.0 |             R |     3072x2048 |
|        190515_Li_00000004.BIAS.fits |    Bias Frame |           0.0 |             R |     3072x2048 |
|        190515_Li_00000005.BIAS.fits |    Bias Frame |           0.0 |             R |     3072x2048 |
|        190515_Li_00000006.BIAS.fits |    Bias Frame |           0.0 |             R |     3072x2048 |
|        190515_Li_00000007.BIAS.fits |    Bias Frame |           

|        190515_Li_00000099.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000100.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000101.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000102.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000103.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000104.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000105.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000106.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000107.BIAS.fits |    Bias Frame |           0.0 |             V |      1024x682 |
|        190515_Li_00000108.BIAS.fits |    Bias Frame |         